# ACMA Scrapper
##### ©Haris Hassan

In [23]:
##=============================================================================
## ACMAscrapper V4.0.2
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#Author Haris Hassan
#Email haris.hassan@radhaz.com.au 
#linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================
# Import libraries
import re
import requests
import string
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)
#Reset index of dataframe after sorting
UrlReqHeaders = requests.utils.default_headers()
UrlReqHeaders.update(
    {
        'User-Agent': 'My User Agent 1.0',
    }
)

In [2]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/rrl/site_search.site_lookup?pSITE_ID=202460'

In [4]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')

try:
    SiteDetailsTitle = [td.text for td in soup.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)')]
    SiteDetails = [td.text.strip() for td in soup.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
except Exception as e:
    print (e)
    pass

In [5]:
SiteDetailsDictionary = {SiteDetailsTitle[i]: SiteDetails[i] for i in range(len(SiteDetailsTitle))}
SiteDetailsDictionary['Location'] = ' '.join(SiteDetailsDictionary['Location'].split())

#SiteData = pd.DataFrame([SiteDetailsDictionary])
#display(SiteDetailsDictionary['Location'])

In [6]:
#Check if there's another Page
NextPageLink = ''.join(['https://web.acma.gov.au' + x for x in  [tl.get('href') for tl in soup.findAll('a',{'title':"Next Page"})]])

In [7]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [8]:
#Next Page
if NextPageLink and NextPageLink.strip():
    ACMApagep2 = requests.get(NextPageLink)
    soupp2 = BeautifulSoup(ACMApagep2.text, 'lxml')

    ACMAtablep2 = soupp2.find('table',{"class": "tablelist responsive"})
    ACMAheadersp2 = [td.text.strip() for td in ACMAtablep2.select('th')]
    ACMAdatap2 = pd.DataFrame(columns = ACMAheadersp2)

    ACMAlinksp2 = []
    for j in ACMAtablep2.find_all('tr')[1:]:
        ACMAValuesp2 = [tv.text.strip() for tv in j.find_all('td')]
        ACMAdatap2.loc[len(ACMAdatap2)] = ACMAValuesp2
        ACMAlinksp2.append([tl.get('href') for tl in j.find_all('a')][0])
    ACMAdatap2.insert(1,'links', ACMAlinksp2 )

    ACMAdata_final = pd.concat([ACMAdata, ACMAdatap2], axis=0)
else:
    ACMAdata_final = ACMAdata
#Reset index of dataframe after sorting
ACMAdata_final = ACMAdata_final.reset_index(drop=True)

In [9]:
#removing duplicates
ACMAdatalinks = list( dict.fromkeys(ACMAdata_final['links']) )
display(ACMAdata_final)

,ID,links,Frequency,Emission Designator,Azimuth,T/R,Client,BSL/Licence No
0,9049910,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9049910,1.9724 GHz,3M84G7W--,40°,R,Vodafone Australia Pty Limited (1103274),10143136
1,9049928,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9049928,1.9772 GHz,3M99G7W--,40°,R,Vodafone Australia Pty Limited (1103274),10143136
2,9049929,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9049929,1.9772 GHz,3M99G7W--,120°,R,Vodafone Australia Pty Limited (1103274),10143136
3,9098008,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9098008,2.1672 GHz,3M99G7W--,120°,T,Vodafone Australia Pty Limited (1103274),10143136
4,9530776,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9530776,827.5 MHz,5M00W7D,200°,R,Vodafone Hutchison Australia Pty Limited (1136980),9263429
...,...,...,...,...,...,...,...,...
147,11596279,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11596279,3.6675 GHz,65M0W7D,200°,T,MOBILE JV PTY LIMITED (20037707),10917464
148,11596287,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=11596287,3.6675 GHz,65M0W7D,200°,R,MOBILE JV PTY LIMITED (20037707),10917464
149,9889575,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9889575,1.7325 GHz,15M0W7D,40°,R,Vodafone Australia Pty Limited (1133304),9263452
150,9889585,/rrl/assignment_search.lookup?pDEVICE_REGISTRATION_ID=9889585,1.7325 GHz,15M0W7D,120°,R,Vodafone Australia Pty Limited (1133304),9263452


In [10]:
AntennaTableHeader = ['Licence Number', 'Antenna', 'Client','Device Type','Emission Center Frequency','Transmitter Power',
                      'Antenna Height (AGL)', 
                      'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt','Device Registration ID','Date Authorised',
                      'Destination Link']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)
NotFoundLinks = []

for acmalink in ACMAdatalinks:
    try:
        url = 'https://web.acma.gov.au'+ acmalink
        page = requests.get(url, headers=UrlReqHeaders)
        soup2 = BeautifulSoup(page.text, 'lxml')

        try:
            Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
            AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
        except Exception as e:
            print (e)
            continue

        LinkToList = []
        LinkToTable = soup2.find("table", {"class": "tablelist linked-responsive"})
        if LinkToTable:
            for j in LinkToTable.find_all('tr')[1:]:
                LinkToList.append([tv.text.strip() for tv in j.select('td:nth-of-type(5)')] )

        if url[-2:]=='/1':
            try:
                Antennaheaders2 = [td.text for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(1)') if not td.has_attr('colspan')]
                AntennaValues2 = [td.text.strip() for td in soup2.select_one('table:nth-of-type(2)',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
            except Exception as e:
                print(e)
                print(acmalink)
                continue

        if not LinkToList:
            AntennaValues.extend(['n/a'])
            if url[-2:]=='/1':
                AntennaValues2.extend(['n/a'])

        if LinkToList:
            LinkToListTemp = []
            for x in LinkToList:
                for j in x:
                    LinkToListTemp.append([' '.join(p) for p in [j.split()[-5:]]])
            LinkToListTemp=list(dict.fromkeys([''.join(p) for p in LinkToListTemp]))
            for todel in LinkToListTemp:
                if SiteDetailsDictionary['Location'] in todel:
                    LinkToListTemp.remove(todel)
            AntennaValues.extend([str(LinkToListTemp)])
            if url[-2:]=='/1':
              AntennaValues2.extend([str(LinkToListTemp)])
        Antennaheaders.extend(['Destination Link'])
        if url[-2:]=='/1':
          Antennaheaders2.extend(['Destination Link'])


        AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
        AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
        AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)

        if url[-2:]=='/1':
            AntennaTable_toAdd2 = pd.DataFrame([{Antennaheaders2[i]: AntennaValues2[i] for i in range(len(Antennaheaders2))}])
            AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd2], ignore_index=True)
    except Exception as e:
        print(e)
        print(url)
        NotFoundLinks.append(url)
        continue    

In [11]:
#Formatting Data
AntennaTable = AntennaTable.fillna('')
AntennaTable['Antenna']=([''.join(x[2].title() +' '+ x[1] + ' '+ x[0].title()) for x in AntennaTable['Antenna'].str.split(',', 2)])

if 'EFL ID' in AntennaTable:
    AntennaTable['Device Registration ID'] = AntennaTable['Device Registration ID'].astype(str) + AntennaTable['EFL ID']


In [12]:
  
AntennaTable['Antenna']= AntennaTable['Antenna'].str.strip()
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Rf Industries', 'RFI')
AntennaTable['Antenna']= AntennaTable['Antenna'].str.replace('Parallel Array Of Vertical Dipoles', 'Vertical Dipole Array')

AntennaTable['Client']=AntennaTable['Client'].apply(lambda x: x.title())
AntennaTable['Client']=AntennaTable['Client'].str.replace('Limited', 'Ltd')
AntennaTable['Client']=AntennaTable['Client'].str.replace('New South Wales Government Telecommunications Authority', 'NSWTA')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('/', '-')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jan', 'January')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Feb', 'February')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Mar', 'March')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Apr', 'April')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jun', 'June')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Jul', 'July')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Aug', 'August')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Sep', 'September')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Oct', 'October')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Nov', 'November')
AntennaTable['Date Authorised']=AntennaTable['Date Authorised'].str.replace('Dec', 'December')


In [13]:
#Frequency conversion to MHZ
#freq_list = [str(float(x[0])*1000) + ' MHz' if x[1] == 'GHz' else x[0] + ' MHz' for x in AntennaTable['Emission Center Frequency'].str.split(' ')]
#freq_df = pd.DataFrame({'Freq (MHz)': freq_list})
#AntennaTable = pd.concat([AntennaTable, freq_df], axis=1)
#display(AntennaTable['Freq (MHz)'])

In [14]:


Renamed_headers = ['Licence', 'Antenna', 'Client','Type','Frequency','Power','Height', 
                      'Polarisation','Azimuth', 'Tilt','Device ID','Date Authorised','Destination Link']

AntennaTable_export = AntennaTable
AntennaTable_export = AntennaTable_export.rename(columns={'Licence Number': 'Licence', 'Device Type': 'Type', 
                                                          'Emission Center Frequency': 'Frequency'
                                                          ,'Transmitter Power': 'Power'
                                                          , 'Antenna Height (AGL)': 'Height', 
                                                          'Antenna Polarisation': 'Polarisation', 
                                                          'Antenna Azimuth': 'Azimuth', 
                   'Device Registration ID': 'Device ID','Antenna Tilt': 'Tilt'})

#AntennaTable_export = AntennaTable_export.rename(columns= dict(zip(Antennaheaders, Renamed_headers)))


AntennaTable_export = AntennaTable_export.reset_index(drop=True)
AntennaTable_export.index += 1
display(AntennaTable_export[Renamed_headers])


,Licence,Antenna,Client,Type,Frequency,Power,Height,Polarisation,Azimuth,Tilt,Device ID,Date Authorised,Destination Link
1,10143136,Kathrein 742215 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Receiver,1.9724 GHz,,35 m,Slant,40°,-2°,9049910,06-Dec-2010,n/a
2,10143136,Kathrein 742213 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Receiver,1.9772 GHz,,35 m,Slant,40°,-2°,9049928,19-Aug-2010,n/a
3,10143136,Kathrein 742213 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Receiver,1.9772 GHz,,35 m,Slant,120°,-2°,9049929,19-Aug-2010,n/a
4,10143136,Kathrein 742213 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Transmitter,2.1672 GHz,0 W,35 m,Slant,120°,-2°,9098008,21-Jan-2010,n/a
5,9263429,Tongyu 609017/Q172717 Panel (1 Sector)-R,Vodafone Hutchison Australia Pty Ltd,Receiver,827.5 MHz,,35 m,Slant,200°,-3°,9530776,14-Dec-2014,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,10917464,Nokia AEQE_I/CS7801002_IPAA_2.7m_3300-3800 Panel(1Sector),Mobile Jv Pty Ltd,Transmitter,3.6675 GHz,325 W pY,35 m,Slant,200°,0°,11596279,07-Feb-2022,n/a
145,10917464,Nokia AEQE_I/CS7801002_IPAA_2.7m_3300-3800 Panel(1Sector),Mobile Jv Pty Ltd,Receiver,3.6675 GHz,,35 m,Slant,200°,0°,11596287,07-Feb-2022,n/a
146,9263452,Kathrein 742213 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Receiver,1.7325 GHz,,35 m,Slant,40°,0°,9889575,22-Jan-2016,n/a
147,9263452,Kathrein 742213 Panel (1 Sector)-R,Vodafone Australia Pty Ltd,Receiver,1.7325 GHz,,35 m,Slant,120°,0°,9889585,22-Jan-2016,n/a


In [34]:
display(NotFoundLinks)

[]

In [33]:
AntennaTable_export.to_excel(r'C:\Users\Mewtwo\Desktop'+'\\ACMA '+  re.sub("[\\\\/]", " ", AntennaDictionary['Site']) + '.xlsx', columns = Renamed_headers, index=True)

In [15]:
#AntennaTable_export.to_html(r'C:\Users\Mewtwo\Desktop\Antennadata.html', columns = Renamed_headers, index=True)
#import subprocess
#subprocess.call('wkhtmltoimage -f png --width 0 Antennadata.html Antennadata.png', shell=True)

1